In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [ ]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.head()

In [ ]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

In [12]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
FL    60040
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
IN    16601
MO    16350
LA    13668
OR    13456
UT     9295
IA     8295
NV     8262
NE     5804
MS     5502
HI     4980
NM     4485
ID     4416
RI     3792
ND     3222
MT     3144
PR     2952
OK     2839
SD     2695
SC     2335
WY     2139
KS     1645
GU      443
VI      290
MP       82
NC       11
AL        8
PA        8
AK        5
AZ        4
MA        3
NJ        3
TN        3
IL        2
ME        2
AR        2
OH        1
GA        1
WV        1
CT        1
Name: BorrowerState, dtype: int64

In [ ]:
ppp_togeoKS = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "KS"]
ppp_togeoKS #8202

In [ ]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoKS_add = ppp_togeoKS[cols]
ppp_togeoKS_add

In [ ]:
add = ppp_togeoKS["full_add"].astype(str).to_list()

In [ ]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #{KS 2-3 Digit Longitude}
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('20')] #KS 
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1

In [ ]:
print(len(lat))
print(len(long))
print(len(geo_id))

In [ ]:
pd.set_option('display.max_rows', None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df

In [ ]:
#pd.set_option('display.max_rows', None)
test_loc = lat_df
test_loc = test_loc.sort_values(["Lat"], ascending=[True], ignore_index=False)
test_loc

In [ ]:
#test_loc.loc[test_loc.index[230], "Lat"] = "37.753937"

In [ ]:
test_loc_drop = test_loc.drop(index=[
5513, 431, 2266, 2887,3127,2971,1500,4121,5700,8239,
5142,7847,7849,6572,6016,6822,762,3630,3914,5181,3187,3864,5866,
4261,5395,4596,3485,1371,1353,8089,8081,3330,471,4463,7012,7015,5694,
644,5173,8356,6112,8046,8048,7953,5646,7943,6034,6036,8325,1657,
353,5612,6446,4982,6551,4451,4484,463,4770,7285,7963,1518,5723,754,1342,
6290,3834,2372,4886,8383,8381,6885,1974,2258,1824,2754,8351,1426,995,5954,5468,
1305,7147,7143,4627,1402,2706,5266,922,6759,6659,1554,4012,4675,7116,5745,4111,1551,7399,4619,6503,
5568,7704,6129,6131,5793,28,7677,2603,4254,4884,2632,2010,3403,3153,3549,5703,5919,7614,7898,8314,4563,4561,7347,
965,633,2555,1366,7291,7080,4124,1398,5030,4875,346,3237,777,6753,7326,4984,5008,7001,7617,2437,2088,2052,7391,
6913,7340,6825,7655,5404,5478,7003,2218,3878,4633,8085,7843,1117,7510,2393,2161,5758,
5625,7711,5200,1438,7019,7010,1168,1143,6527,4973,4938,4920,1900,1902,987,5417,5413,
1145,1147,6278,7202,1092,6600,3845,7324,7931,110,8389,1288,1534,371,286,2244,8037,
4642,6657,7977,4569,3383,8255,8252,1957,1971,8213,4810,6168,7550,2177,5680,4740,
46,4525,5676,5673,5501,6030,8511,6190,5967,3417,7764,6068,5934,665,7629,6698,
1703,1930,8328,2585,3143,8513,8509,7801,64,4637,6055,3323,7884,7403,6153,4791,
4044,4653,2027,2062,3920,4836,4712,4434,3513,6802,4431,8444,5903,4898,5916,3089,
1585,2804,2774,2744,2725,2752,2528,2534,5282,185,4137,3372,3370,1179,3671,6740,1213,8265,
1955,435,2303,1467,5812,4512,6641,8291,485,7603,4325,3297,7356,2577,2579,
7445,297,3739,7165,6725,377,7591,571,7052,8305,5011,5476,4706,2540,6382,
4441,960,7342,4059,4061,2902,8284,6574,2388,8353,955,4003,786,
6499,1391, 3290])
test_loc_drop 

In [ ]:
test_resort =test_loc_drop.sort_index()
test_resort

In [ ]:
test_resort_reset = test_resort.reset_index()

In [ ]:
test_resort_reset

In [ ]:
lat_list = test_resort_reset["Lat"].to_list()

In [ ]:
result_df = pd.DataFrame({"full_add":add, "Lat":lat_list, "Long":long, "FIPS_z": geo_id})
result_df 

In [ ]:
result_df.loc[result_df.index[225], "Lat"] = "37.753937"

In [ ]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

In [ ]:
ppp_togeoKS_merge = ppp_togeoKS.merge(result_df, on="full_add", how ="left")
ppp_togeoKS_merge["Lat"] = ppp_togeoKS_merge["Lat_y"]
ppp_togeoKS_merge["Long"] = ppp_togeoKS_merge["Long_y"]
ppp_togeoKS_merge["FIPS_z"] = ppp_togeoKS_merge["FIPS_z_y"]
ppp_togeoKS_merge = ppp_togeoKS_merge.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_x", "FIPS_z_y"], axis=1)
ppp_togeoKS_merge


In [76]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoKS_parsed = ppp_togeoKS_merge[cols]
ppp_togeoKS_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
0,2766427408,200450006021015,38.953377,-95.30946
1,3354047105,201690001023010,38.84512,-97.60422
2,5028487210,201730095132002,37.723057,-97.440765
3,9101897101,200510729013036,38.87494,-99.332695
4,9638147110,None,None,None


In [77]:
ppp_geo = ppp_geo.merge(ppp_togeoKS_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat_x,Long_x,FIPS_z_x,FIPS_z_y,Lat_y,Long_y
0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,NaN,NaN,NaN
1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,NaN,NaN,NaN
2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,NaN,NaN,NaN
3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN,NaN,NaN,NaN
4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,NaN,NaN,NaN


In [78]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [79]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo_cens.csv")